In [1]:
# %% 1. การติดตั้งและ Import Library

# ติดตั้ง library ที่จำเป็น
!pip install -q transformers[torch] datasets accelerate scikit-learn seqeval

import json
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, Features, Value, ClassLabel, Sequence
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    Trainer,
    TrainingArguments,
    DataCollatorForTokenClassification
)
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report

# ตรวจสอบและตั้งค่าอุปกรณ์ (GPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"อุปกรณ์ที่ใช้: {DEVICE}")


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



อุปกรณ์ที่ใช้: cuda


In [2]:
# %% 2. การโหลดและแปลงข้อมูลดิบ (CORD Dataset)

print("กำลังโหลดและประมวลผลข้อมูล CORD...")
# โหลดข้อมูล CORD จาก Hugging Face Hub (ใช้เฉพาะส่วน train มาสาธิต)
raw_dataset = load_dataset("naver-clova-ix/cord-v2", split="train")

# --- สร้าง Label Mapping ---
# กำหนด Entity ที่เราสนใจจากโจทย์ และสร้าง Label ของเราเอง
# เพิ่ม B- และ I- prefix ตามมาตรฐาน IOB2
labels = ["COMPANY", "DATE", "AMOUNT"]
ner_tags_list = ["O"] + [f"B-{label}" for label in labels] + [f"I-{label}" for label in labels]
label2id = {label: i for i, label in enumerate(ner_tags_list)}
id2label = {i: label for i, label in enumerate(ner_tags_list)}

# Mapping จาก Label เดิมของ CORD ไปยัง Label ใหม่ของเรา
cord_label_map = {
    "store_name": "COMPANY",
    "payment_date": "DATE",
    "total_price": "AMOUNT"
}

def process_cord_example(example):
    """
    ฟังก์ชันสำหรับแปลง 1 ตัวอย่างข้อมูลจาก CORD ให้อยู่ในรูปแบบ tokens และ ner_tags
    """
    # โหลด ground_truth ที่เป็น JSON string
    ground_truth = json.loads(example["ground_truth"])
    
    words = []
    tags = []
    
    for item in ground_truth["valid_line"]:
        for word_info in item["words"]:
            text = word_info["text"]
            category = item["category"]
            
            # แปลง label เดิมของ cord เป็น label ใหม่ของเรา
            mapped_label = "O" # Default คือ O
            if category in cord_label_map:
                mapped_label = cord_label_map[category]

            # แบ่งคำที่มี space ข้างในออกเป็นหลายๆ คำ
            sub_words = text.split()
            if not sub_words:
                continue
                
            # แปะป้าย B- ให้คำแรก และ I- ให้คำที่เหลือ
            words.append(sub_words[0])
            tags.append(f"B-{mapped_label}" if mapped_label != "O" else "O")
            
            for sub_word in sub_words[1:]:
                words.append(sub_word)
                tags.append(f"I-{mapped_label}" if mapped_label != "O" else "O")

    return {"tokens": words, "ner_tags": [label2id[tag] for tag in tags]}

# ใช้ .map() เพื่อประมวลผลข้อมูลทั้งหมด
processed_dataset = raw_dataset.map(process_cord_example, remove_columns=raw_dataset.column_names)

# แบ่งข้อมูลเป็น train และ test set
dataset_dict = processed_dataset.train_test_split(test_size=0.2, seed=42)

print("\nประมวลผลข้อมูลสำเร็จ")
print("ตัวอย่างข้อมูลที่แปลงแล้ว:")
example = dataset_dict["train"][1]
print(f"Tokens: {example['tokens']}")
print(f"Tags  : {[id2label[tag_id] for tag_id in example['ner_tags']]}")


กำลังโหลดและประมวลผลข้อมูล CORD...

ประมวลผลข้อมูลสำเร็จ
ตัวอย่างข้อมูลที่แปลงแล้ว:
Tokens: ['1', 'KFC', 'Winger', 'HC', '20,000', 'Sub', 'Total', '20,000', 'Dasar', 'Pengenaan', 'Pajak', '20,000', 'P.Rest', '10', '%', '2,000', 'Total', '22,000', 'Cash', '22,000', '1', 'Items,']
Tags  : ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [3]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 640
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 160
    })
})

In [4]:
# %% 3. การ Tokenize และจัดเรียง Label

# เนื่องจาก CORD เป็นภาษาอังกฤษ เราจะใช้โมเดล multilingual เพื่อสาธิต
MODEL_NAME = "FacebookAI/xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_and_align_labels(examples):
    """
    ฟังก์ชันสำหรับ Tokenize และจัดเรียง Label ให้ตรงกับ Sub-word
    """
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100) # สำหรับ Special token [CLS], [SEP]
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx]) # สำหรับ sub-word แรกของคำ
            else:
                label_ids.append(-100) # สำหรับ sub-word ที่เหลือของคำ
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# ใช้ .map() เพื่อ Tokenize ข้อมูลทั้งหมด
tokenized_datasets = dataset_dict.map(tokenize_and_align_labels, batched=True)

print("\nTokenize และจัดเรียง Label สำเร็จ")
print("ตัวอย่างข้อมูลที่ผ่านการ Tokenize:")
print(tokenized_datasets['train'][0].keys())



Tokenize และจัดเรียง Label สำเร็จ
ตัวอย่างข้อมูลที่ผ่านการ Tokenize:
dict_keys(['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'])


In [5]:
# %% 4. การตั้งค่าสำหรับการ Fine-tune

# โหลดโมเดลสำหรับ Token Classification พร้อมระบุจำนวน Label
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(ner_tags_list),
    id2label=id2label,
    label2id=label2id
).to(DEVICE)

# Data Collator จะช่วยจัดการเรื่อง Padding ให้เราอัตโนมัติ
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# กำหนดฟังก์ชันสำหรับคำนวณ Metrics ด้วย seqeval
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # แปลง ID กลับเป็น Label และลบ -100 ที่เราใส่ไว้
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    return {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
    }

import tempfile
output_dir = tempfile.mkdtemp(prefix="ner_results_")
# กำหนด Training Arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

print("ตั้งค่า Model, Arguments, และ Metrics สำเร็จ")


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ตั้งค่า Model, Arguments, และ Metrics สำเร็จ


c:\Users\potij\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
# import os
# import tempfile

# # วิธีที่ 1: ใช้ temporary directory
# output_dir = tempfile.mkdtemp(prefix="nlp_results_")

# # วิธีที่ 2: หรือใช้ path ที่เรียบง่าย (ถ้าวิธีที่ 1 ไม่ทำงาน)
# # output_dir = r"C:\temp\nlp_results"
# # os.makedirs(output_dir, exist_ok=True)

# print(f"Output directory: {output_dir}")

# # กำหนด Training Arguments
# training_args = TrainingArguments(
#     output_dir=output_dir,
#     num_train_epochs=3,                 # จำนวนรอบในการฝึก
#     per_device_train_batch_size=16,     # ขนาด batch สำหรับ train
#     per_device_eval_batch_size=16,      # ขนาด batch สำหรับ eval
#     warmup_steps=500,                   # จำนวน step สำหรับ warm up learning rate
#     weight_decay=0.01,                  # ค่า weight decay
#     logging_steps=100,
#     evaluation_strategy="epoch",        # ประเมินผลทุกๆ 1 epoch
#     save_strategy="epoch",              # บันทึกโมเดลทุกๆ 1 epoch
#     load_best_model_at_end=True,        # โหลดโมเดลที่ดีที่สุดหลังฝึกเสร็จ
#     metric_for_best_model="f1",         # ใช้ f1 score เป็นเกณฑ์เลือกโมเดลที่ดีที่สุด
# )

In [7]:
# %% 5. การ Fine-tune โมเดล

# สร้าง Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("กำลังเริ่มการ Fine-tune โมเดล...")
# เริ่มฝึก
trainer.train()
print("\nการ Fine-tune เสร็จสิ้น")


C:\Users\potij\AppData\Local\Temp\ipykernel_6388\3817340510.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


กำลังเริ่มการ Fine-tune โมเดล...


wandb: Currently logged in as: potijark5 (potijark5-no). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.000566,0.000000,0.000000,0.000000
2,No log,0.000290,0.000000,0.000000,0.000000
3,No log,0.000256,0.000000,0.000000,0.000000


c:\Users\potij\AppData\Local\Programs\Python\Python312\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\potij\AppData\Local\Programs\Python\Python312\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\potij\AppData\Local\Programs\Python\Python312\Lib\site-packages\seqeval\metrics\v1.py:159: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
c:\Users\potij\AppData\Local\Programs\Python\Python312\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricW


การ Fine-tune เสร็จสิ้น


### Inference

In [8]:
# %% 6. การทดสอบทำนายผล (Inference)

from transformers import pipeline

# สร้าง pipeline สำหรับ Token Classification จากโมเดลที่เราเพิ่งฝึกเสร็จ
# ตั้งค่า aggregation_strategy="simple" เพื่อให้ pipeline รวม sub-word ให้เราอัตโนมัติ
ner_pipeline = pipeline(
    "ner",
    model=trainer.model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    device=DEVICE
)

# ข้อความตัวอย่างที่จำลองมาจากใบแจ้งหนี้
sample_text = "Invoice from Tech Solutions Inc. Date: 15/07/2025. Total Amount is $ 4,500.50"

# ทำนายผล
results = ner_pipeline(sample_text)

print(f"ข้อความ Input: '{sample_text}'")
print("\nผลลัพธ์การสกัดข้อมูล:")
for entity in results:
    print(f"  - Entity: {entity['entity_group']}, Value: {entity['word']}, Score: {entity['score']:.4f}")

# --- เขียนฟังก์ชันง่ายๆ เพื่อแปลงผลลัพธ์เป็น Dictionary ---
def extract_entities_to_dict(ner_results):
    extracted_data = {"COMPANY": None, "DATE": None, "AMOUNT": None}
    for entity in ner_results:
        entity_type = entity['entity_group']
        if entity_type in extracted_data:
            extracted_data[entity_type] = entity['word']
    return extracted_data

final_data = extract_entities_to_dict(results)
print("\nผลลัพธ์ในรูปแบบ Dictionary:")
print(final_data)


Device set to use cuda


ข้อความ Input: 'Invoice from Tech Solutions Inc. Date: 15/07/2025. Total Amount is $ 4,500.50'

ผลลัพธ์การสกัดข้อมูล:

ผลลัพธ์ในรูปแบบ Dictionary:
{'COMPANY': None, 'DATE': None, 'AMOUNT': None}


### Submission 

In [9]:
# %% 7. การสร้างไฟล์ Submission (จำลอง)

print("กำลังจำลองการสร้างไฟล์ submission.csv...")

# สร้าง Test DataFrame จำลอง
test_data = {
    "invoice_id": ["inv_001", "inv_002"],
    "invoice_text": [
        "Receipt from Global Mart LLC, Date: 20/11/2024, FINAL TOTAL: 199.99",
        "Cyber Systems Ltd. billed you 1,250.00 on 01/01/2025"
    ]
}
test_df = pd.DataFrame(test_data)

# วนลูปเพื่อทำนายผลแต่ละรายการ
predictions = []
for text in test_df['invoice_text']:
    ner_results = ner_pipeline(text)
    extracted_data = extract_entities_to_dict(ner_results)
    predictions.append(extracted_data)

# สร้าง DataFrame จากผลลัพธ์
submission_df = pd.DataFrame(predictions)
submission_df.columns = ["company_name", "invoice_date", "total_amount"]
submission_df.insert(0, 'invoice_id', test_df['invoice_id'])


print("\nตัวอย่างข้อมูลในไฟล์ Submission:")
print(submission_df.head())

# บันทึกเป็นไฟล์ CSV
submission_df.to_csv("submission_ner.csv", index=False)

print("\nสร้างไฟล์ submission_ner.csv สำเร็จ!")


กำลังจำลองการสร้างไฟล์ submission.csv...

ตัวอย่างข้อมูลในไฟล์ Submission:
  invoice_id company_name invoice_date total_amount
0    inv_001         None         None         None
1    inv_002         None         None         None

สร้างไฟล์ submission_ner.csv สำเร็จ!


In [10]:
predictions

[{'COMPANY': None, 'DATE': None, 'AMOUNT': None},
 {'COMPANY': None, 'DATE': None, 'AMOUNT': None}]

In [11]:
submission_df

,invoice_id,company_name,invoice_date,total_amount
0,inv_001,None,None,None
1,inv_002,None,None,None
